<a href="https://colab.research.google.com/github/preetamjumech/LLM/blob/main/Exploring_the_depths_of_LLMS_01_10_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install torch

In [2]:
# GPT-2 ia a transformer-based model designed for text generation. It has 3 layers:
# 1. Embedding Layer:
# 2. Transformer Blocks:
# 3. Output Layer:

from transformers import GPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [6]:
text = "Machine learning is"
input_ids = tokenizer.encode(text, return_tensors = "pt")
print(input_ids)
print(input_ids[0])
print(f"Tokens: {input_ids[0].tolist()}")
print(f"Tokenized Text: {tokenizer.convert_ids_to_tokens(input_ids[0])}")

tensor([[37573,  4673,   318]])
tensor([37573,  4673,   318])
Tokens: [37573, 4673, 318]
Tokenized Text: ['Machine', 'Ġlearning', 'Ġis']


In [7]:
import torch
with torch.no_grad():
  output = model(input_ids)
predicted_token_id = torch.argmax(output.logits[:,-1,:]).item()
predicted_token = tokenizer.decode(predicted_token_id)
print(predicted_token)

 a


In [8]:
def generate_text(text, max_length):
  input_ids  = tokenizer.encode(text, return_tensors = "pt")
  output = model.generate(input_ids, max_length = max_length, num_beams = 5, no_repeat_ngram_size = 2, early_stopping = True)
  generated_text = tokenizer.decode(output[0], skip_special_tokens = True)
  return generated_text

prompt = "Machine learning is"
example_generated_text = generate_text(prompt, max_length = 50)
print(example_generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Machine learning is one of the most promising areas of research in the field of artificial intelligence.

In the past few years, researchers have been working on ways to train neural networks to perform tasks that are difficult for humans to do. For example,


# Finetuning LLM on custom data

In [20]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW

class DomainSpecificFineTuner:
  def __init__(self, model_name, train_file, epochs = 1):
    self.tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    self.tokenizer.pad_token = self.tokenizer.eos_token
    self.model = GPT2LMHeadModel.from_pretrained(model_name)
    self.epochs = epochs
    self.train_file = train_file


  class TextDataset(Dataset):
    def __init__(self, tokenizer, file_path, block_size= 128):
      self.tokenizer = tokenizer
      with open(file_path, "r", encoding = "utf-8") as f:
        lines = f.readlines()
        print(lines)
      self.data = tokenizer(lines, truncation = True, padding = True, return_tensors = "pt")

    def __len__(self):
      return len(self.data["input_ids"])

    def __getitem__(self, i):
      return {"input_ids": self.data["input_ids"][i], "attention_mask": self.data["attention_mask"][i]}


  def train(self):
    train_dataset = self.TextDataset(self.tokenizer, self.train_file)
    dataloader = DataLoader(train_dataset, batch_size = 4, shuffle = True)
    optimizer = AdamW(self.model.parameters(), lr = 5e-5)

    for epoch in range(self.epochs):
      for i, batch in enumerate(dataloader):
        outputs = self.model(input_ids = batch["input_ids"], attention_mask = batch["attention_mask"], labels = batch["input_ids"])
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if i % 10 == 0:
          print(f"Epoch: {epoch }, Batch: {i}, Loss: {loss.item()}")

tuner = DomainSpecificFineTuner("gpt2", "test.txt", epochs = 3)
tuner.train()

['### The Space King: Ghost Astronaut\n', '\n', 'Once upon a time, in a galaxy far away, there was a kingdom known as Starhaven. This magical kingdom was not like any ordinary realm; it floated among the stars, shimmering with colors no one had ever seen. The ruler of Starhaven was a kind and brave king named Astraeus, who had a deep love for the cosmos.\n', '\n', 'King Astraeus had a special gift: he could communicate with the stars and understand their secrets. He often ventured into the depths of space in his shining silver spaceship, the Celestial Voyager, to explore distant planets and meet extraordinary beings. His adventures were legendary, and he collected stories and treasures from every corner of the universe.\n', '\n', 'But one fateful day, during an expedition to the mysterious Nebula of Whispers, Astraeus encountered a dark force. A swirling storm appeared out of nowhere, and his ship was caught in its grip. Despite his bravery, Astraeus couldn’t escape. The storm swallowe

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch: 0, Batch: 0, Loss: 6.167759418487549
Epoch: 1, Batch: 0, Loss: 1.2773746252059937
Epoch: 2, Batch: 0, Loss: 1.5982493162155151


In [16]:
 with open(r"/content/train.txt", "rb") as f:
        lines = f.readlines()

print(lines)

[b'powering progress process engineering value proposition preetam research alliance setting research collaboration external partner evren unsal academic partnership manager netherlands november term definition november external technology collaboration etc research project external partners universities companies governmental institutes etna external technology database online database manage etc request preetam scientist technologists salesforce platform type etcs supported etna bilateral collaboration project one partner only partner academic industrial governmental research institute joint industrial project jip project multiple partners typically industrial companies university also member always lead partner coordinate project partnership government industry funding together subsidized project project receive public funding agreed effort proposal common public funding agency european union commission department energy doe usa membership joining research program consortium fee typ

# Question Answering

In [21]:
def answer_question(model, tokenizer, context, question, max_length=50):
    input_text = context + "\n" + question
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    model.eval()  # Set the model to evaluation mode

    with torch.no_grad():
        output_sequence = model.generate(
            input_ids=input_ids,
            max_length=max_length,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            temperature=0.7,
            top_k=50,
            top_p=0.95,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

    output_text = tokenizer.decode(output_sequence[0], skip_special_tokens=True)
    return output_text

# Example usage
context = "The Space King Astraeus could communicate with the stars and had adventures across the galaxy."
question = "What could Astraeus do?"
answer = answer_question(tuner.model, tuner.tokenizer, context, question)
print(f"Question: {question}\nAnswer: {answer}\n")


Question: What could Astraeus do?
Answer: The Space King Astraeus could communicate with the stars and had adventures across the galaxy.
What could Astraeus do?



# Text Generation

In [22]:
def generate_text(model, tokenizer, prompt, max_length=50, num_return_sequences=1):
    model.eval()  # Set the model to evaluation mode
    input_ids = tokenizer.encode(prompt, return_tensors='pt')

    with torch.no_grad():
        output_sequences = model.generate(
            input_ids=input_ids,
            max_length=max_length,
            num_return_sequences=num_return_sequences,
            no_repeat_ngram_size=2,
            temperature=0.7,
            top_k=50,
            top_p=0.95,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

    generated_texts = []
    for generated_sequence in output_sequences:
        text = tokenizer.decode(generated_sequence, skip_special_tokens=True)
        generated_texts.append(text)

    return generated_texts

# Example usage
tuner.model.eval()  # Make sure to set the model to evaluation mode
prompt = "Once upon a time in a distant galaxy"
generated_texts = generate_text(tuner.model, tuner.tokenizer, prompt, max_length=50)
for idx, text in enumerate(generated_texts):
    print(f"Generated Text {idx + 1}:\n{text}\n")


Generated Text 1:
Once upon a time in a distant galaxy, a group of mysterious beings known as the Red Lotus were struggling to find a way to stop their evil, and they were able to harness the power of the Force. They had been trapped in their mysterious form

